In [1]:
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="malaysia-ai/Multilingual-TTS", 
#     repo_type="dataset",
#     allow_patterns="*/*.parquet",
#     local_dir="./Multilingual-TTS",
# )

In [2]:
# snapshot_download(
#     repo_id="malaysia-ai/Multilingual-TTS", 
#     repo_type="dataset",
#     allow_patterns="*_neucodec.zip",
#     local_dir="./",
# )

In [3]:
from glob import glob
import pandas as pd
import json
import os
import torch
import IPython.display as ipd

torch.set_grad_enabled(False)

from transformers import AutoTokenizer, AddedToken
from streaming import MDSWriter
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import LocalDataset
import numpy as np
from tqdm import tqdm
from multiprocess import Pool
import itertools

def chunks(l, n):
    for i in range(0, len(l), n):
        yield (l[i: i + n], i // n)

def multiprocessing(strings, function, cores=6, returned=True):
    df_split = chunks(strings, len(strings) // cores)
    pool = Pool(cores)
    pooled = pool.map(function, df_split)
    pool.close()
    pool.join()

    if returned:
        return list(itertools.chain(*pooled))

class UInt32(Encoding):
    def encode(self, obj) -> bytes:
        return obj.tobytes()

    def decode(self, data: bytes):
        return np.frombuffer(data, np.uint32)

_encodings['uint32'] = UInt32

columns = {
    'input_ids': 'uint32',
    'position_ids': 'uint32',
    'attention_mask': 'uint32',
    'audio': 'str',
    'text': 'str'
}
hashes = 'sha1', 'xxh64'

def new_path(f):
    splitted = f.split('/')
    folder = f.split('/')[0]
    folder = folder + '_neucodec'
    new_f = os.path.join(folder, '/'.join(splitted[1:]))
    new_f = new_f.replace('.mp3', '.json').replace('.wav', '.json')
    return new_f

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def loop(files):
    files, _ = files
    data = []
    for f in tqdm(files):
        df = pd.read_parquet(f).to_dict(orient = 'records')
        for i in range(len(df)):
            token_filename = new_path(df[i]['audio_filename'])
            if not os.path.exists(token_filename):
                continue
            df[i]['token_filename'] = token_filename
            data.append(df[i])
    return data

In [ ]:
files = glob('Multilingual-TTS/*/*.parquet')
files = [f for f in files if 'Malaysian-TTS-v2' not in f]
data = multiprocessing(files, loop, cores = 20)
len(data)

In [ ]:
malaysian = pd.read_parquet('Multilingual-TTS/Malaysian-TTS-v2/train-00000-of-00001.parquet').to_dict(orient = 'records')

In [5]:
tokenizer = AutoTokenizer.from_pretrained('Qwen/Qwen3-1.7B-Base')
extra = [AddedToken('<|speech_start|>')]
for i in range(65536):
    extra.append(AddedToken(f'<|s_{i}|>'))
tokenizer.add_tokens(extra)

65537

In [ ]:
import gc

def collator(batch, batch_position_ids):
    input_ids = []
    position_ids = []
    masks = []
    for i in range(len(batch)):
        l = len(batch[i])
        input_ids.extend(batch[i])
        position_ids.extend(batch_position_ids[i])
        masks.append(l)
    
    return {
        'input_ids': np.array(input_ids).astype(np.uint32),
        'position_ids': np.array(position_ids).astype(np.uint32),
        'attention_mask': np.array(masks).astype(np.uint32),
        'audio': '',
        'text': '',
    }

def slice_and_balance(nested_list, size):
    first = []
    balance = []
    current_size = 0

    for sublist in nested_list:
        if current_size < size:
            remaining_space = size - current_size
            if len(sublist) <= remaining_space:
                first.append(sublist)
                current_size += len(sublist)
            else:
                first.append(sublist[:remaining_space])
                balance.append(sublist[remaining_space:])
                current_size = size
        else:
            balance.append(sublist)
    
    return first, balance

In [ ]:
!rm -rf tokenized-4k-qwen3

In [ ]:
import time

sequence_length = 1024 * 10
def loop(files, block_size = sequence_length):
    rows, index = files
    out_root = f'tokenized-4k-qwen3/tokenized-{index}'
    os.system(f'rm -rf {out_root}')
    count = 0
    temp = []
    position_ids = []
    last_block, last_position_block = None, None
    with MDSWriter(out=out_root, columns=columns, compression=None, hashes=hashes) as out:
        for row in tqdm(rows):

            text = row['text']
            
            try:
                with open(row['token_filename']) as fopen:
                    token = json.load(fopen)
            except:
                continue

            if len(text.split()) > len(token):
                continue

            left = row['speaker'] +': ' + text
            
            token = ''.join([f'<|s_{t}|>' for t in token])
            prompt = f'<|im_start|>{left}<|speech_start|>{token}<|im_end|>'
            
            outputs = tokenizer(prompt, add_special_tokens = False)
            position = range(len(outputs['input_ids']))
            length = len(outputs['input_ids'])
            
            if count + length > block_size:
                o = collator(temp, position_ids)
                if o['input_ids'].shape[0] > 0:
                    out.write(o)
                temp = [outputs['input_ids']]
                position_ids = [position]
                count = length
                
            else:
                temp.append(outputs['input_ids'])
                position_ids.append(range(len(outputs['input_ids'])))
                count += len(outputs['input_ids'])
        
        if len(temp):
            o = collator(temp, position_ids)
            if o['input_ids'].shape[0] > 0:
                out.write(o)
            

In [ ]:
# loop((data[:10], 0))

In [ ]:
combined = data + malaysian

In [ ]:
multiprocessing(combined, loop, cores = 40, returned = False)

In [6]:
folders = sorted(glob('tokenized-4k-qwen3/tokenized-*'), key = lambda x: int(x.split('-')[-1]))
folders

['tokenized-4k-qwen3/tokenized-0',
 'tokenized-4k-qwen3/tokenized-1',
 'tokenized-4k-qwen3/tokenized-2',
 'tokenized-4k-qwen3/tokenized-3',
 'tokenized-4k-qwen3/tokenized-4',
 'tokenized-4k-qwen3/tokenized-5',
 'tokenized-4k-qwen3/tokenized-6',
 'tokenized-4k-qwen3/tokenized-7',
 'tokenized-4k-qwen3/tokenized-8',
 'tokenized-4k-qwen3/tokenized-9',
 'tokenized-4k-qwen3/tokenized-10',
 'tokenized-4k-qwen3/tokenized-11',
 'tokenized-4k-qwen3/tokenized-12',
 'tokenized-4k-qwen3/tokenized-13',
 'tokenized-4k-qwen3/tokenized-14',
 'tokenized-4k-qwen3/tokenized-15',
 'tokenized-4k-qwen3/tokenized-16',
 'tokenized-4k-qwen3/tokenized-17',
 'tokenized-4k-qwen3/tokenized-18',
 'tokenized-4k-qwen3/tokenized-19',
 'tokenized-4k-qwen3/tokenized-20',
 'tokenized-4k-qwen3/tokenized-21',
 'tokenized-4k-qwen3/tokenized-22',
 'tokenized-4k-qwen3/tokenized-23',
 'tokenized-4k-qwen3/tokenized-24',
 'tokenized-4k-qwen3/tokenized-25',
 'tokenized-4k-qwen3/tokenized-26',
 'tokenized-4k-qwen3/tokenized-27',
 '

In [8]:
!rm -rf multipacking

In [9]:
with MDSWriter(out='multipacking', columns=columns, compression=None, hashes=hashes) as out:
    for f in folders:
        try:
            dataset = LocalDataset(local=f)
            for i in tqdm(range(len(dataset))):
                out.write(dataset[i])
        except Exception as e:
            print(e)
            pass

100%|██████████| 1/1 [00:00<00:00, 3123.09it/s]


In [10]:
dataset = LocalDataset('multipacking')
len(dataset)

507262

In [ ]:
# tokenizer.decode(dataset[255075]['input_ids'])

In [ ]:
# for i in tqdm(range(len(dataset))):
#     if dataset[i]['attention_mask'].max() > 9000:
#         print(i, dataset[i]['attention_mask'])

In [ ]:
dataset[0]['attention_mask'].max()

In [ ]:
s = tokenizer.decode(dataset[255076]['input_ids'])

In [ ]:
splitted = s.split('<|im_end|>')[0].split('<|speech_start|>')
splitted

In [ ]:
import re

numbers = re.findall(r's_(\d+)', splitted[1])
d = list(map(int, numbers))
len(splitted[0].split()), len(d)

In [ ]:
(len(d) / 50) / len(splitted[0])

In [ ]:
from neucodec import NeuCodec
import torch

neucodec = NeuCodec.from_pretrained("neuphonic/neucodec")
_ = neucodec.eval().cuda()

In [ ]:
fsq_codes = torch.tensor(d)[None][None]
recon = neucodec.decode_code(fsq_codes.cuda()).cpu()

In [ ]:
import IPython.display as ipd
ipd.Audio(recon[0, 0], rate = 24000)

In [ ]:
# !hf upload malaysia-ai/multipacking-multilingual-tts-10k-qwen3 multipacking --repo-type=dataset